# Stocks Converter

## Define the converter

In [7]:
import numpy as np

In [44]:
class StockData:
    """convert real-world stock data to annualized covariance matrix and annualized return
    input: a list of continuous stock data in the same time span
    output: annualized convariance matrix and return
    """

    def __init__(self, data):
        self.data = data  # add data
        self.n_stocks = len(data)  # num of stocks

        # check the number of days
        n_days = [len(i) for i in data]
        if max(n_days) != (sum(n_days) / len(n_days)):
            raise Exception("timespan of stocks should be the same")
        self.n_days = len(data[1])

        # calculate the daily percentage price change
        self.daily_change = []  # daily percentage price change
        for i in range(self.n_stocks):
            each_stcok = []
            for j in range(self.n_days - 1):
                each_stcok.append((data[i][j + 1] - data[i][j]) / data[i][j])
            self.daily_change.append(each_stcok)

    # calculate annualized return (mu)
    def get_return(self):
        change = [[i + 1 for i in j] for j in self.daily_change]  # daily_change + 1
        ret = [np.prod(i) ** (252 / self.n_days) for i in change]
        return ret

    # calculate annualized covariance matrix (sigma)
    def get_covariance(self):
        mean_change = [
            [i - np.mean(j) for i in j] for j in self.data
        ]  # daily_change - mean(daily_change)
        cov = 252 / self.n_days * np.dot(mean_change, np.transpose(mean_change))
        return cov

    def get_pentalty(self, cov, ret, risk_pre, budget):
        # get all fesible and unfeasible states
        self.f_state = []  # feasible states (num of '1's equal to budge)
        self.uf_state = []  # unfeasible states
        self.all_state = []
        for i in range(2**self.n_stocks):
            state = f"{bin(i)[2:]:0>{self.n_stocks}}"
            n_ones = 0
            for j in state:
                if j == "1":
                    n_ones += 1
            self.all_state.append(state)
            if n_ones == budget:
                self.f_state.append(state)
            else:
                self.uf_state.append(state)

        # determine the penalty factor
        mark = False
        penalty = 0  # initial value
        while mark == False:
            R = np.diag(ret)
            S = np.ones((self.n_stocks, self.n_stocks)) - 2 * budget * np.diag(
                np.ones(self.n_stocks)
            )
            Q = risk_pre * cov - R + penalty * S
            F = []
            for state in self.f_state:
                x = np.array([int(bit) for bit in state])
                F.append(np.dot(x, np.dot(Q, x)) + penalty*budget**2)
            Fmin = np.amin(F)
            Fbar = np.mean(F)
            F = []
            for state in self.uf_state:
                x = np.array([int(bit) for bit in state])
                F.append(np.dot(x, np.dot(Q, x)) + penalty*budget**2)
            Fmin_uf = np.amin(F)
            location = np.where(F == Fmin_uf)[0][0]
            if Fmin_uf < 0.5 * (Fmin + Fbar):
                n_ones = 0
                for j in self.uf_state[location]:
                    if j == "1":
                        n_ones += 1
                penalty += (0.5 * (Fmin + Fbar) - Fmin_uf) / (n_ones - budget) ** 2
                #mark = True
            else:
                mark = True  # ready to return the penalty
        return penalty

## Import data

In [45]:
import pandas as pd
aapl = pd.read_csv('AAPL.csv')
amzn = pd.read_csv('AMZN.csv')
meta = pd.read_csv('META.csv')
msft = pd.read_csv('MSFT.csv')
qcom = pd.read_csv('QCOM.csv')
sbux = pd.read_csv('SBUX.csv')
names = [aapl, amzn, meta, msft, qcom, sbux]
data = []
for i in names:
    i['Close/Last'].to_string
    data.append([float(j[2::]) for j in i['Close/Last'].transpose()])
data
stock_data = StockData(data)
ret = stock_data.get_return()
cov = stock_data.get_covariance()
stock_data.get_pentalty(cov, ret, 0.5, 3)

80299.22578764349

# Classical

In [24]:
states = []
for i in range(2**6):
    a = f"{bin(i)[2:]:0>{6}}"
    n_ones = 0
    for j in a:
        if j == '1':
            n_ones += 1
    if True:#n_ones != 3:
        states.append(a)

In [25]:
def QUBO_from_portfolio(cov, mean, q, B, t):
    """convert portfolio parameters to a Q-matrix
    cov: n-by-n covariance numpy array
    mean: numpy array of means
    q: the risk preference of investor
    B: budget
    t: penalty factor
    """
    n = cov.shape[0]
    R = np.diag(mean)
    S = np.ones((n, n)) - 2 * B * np.diag(np.ones(n))

    Q = q * cov - R + t * S
    return Q

In [26]:
Q = QUBO_from_portfolio(cov, ret, 0.5, 3, 0)

In [27]:
# Brutely search over classical results for comparison before we run QAOA
# the results are sorted with cost
cost_dict = {}
i = 0
for selection in states:
    x = np.array([int(bit) for bit in selection])
    cost_dict[selection] = np.dot(x, np.dot(Q, x))
    i += 1
cost_sorted = dict(sorted(cost_dict.items(), key=lambda item: item[1]))
print("\n-------------------------------------")
print("    selection\t  |\t  cost")
print("-------------------------------------")
for k, v in cost_sorted.items():
    print("%10s\t  |\t%.4f" % (k, v))
print("-------------------------------------")


-------------------------------------
    selection	  |	  cost
-------------------------------------
    000000	  |	0.0000
    000001	  |	984.3719
    010000	  |	15241.6123
    010001	  |	16607.1970
    000010	  |	17903.0318
    000011	  |	18095.8302
    100000	  |	22076.2938
    100001	  |	24795.6553
    100010	  |	45222.4873
    100011	  |	47150.2753
    010010	  |	51014.8211
    010011	  |	51588.8322
    110000	  |	53745.7751
    110001	  |	56846.3493
    000100	  |	59615.4242
    000101	  |	60211.1503
    001000	  |	84039.2013
    001001	  |	85778.3771
    010100	  |	91550.9992
    010101	  |	92527.9380
    100100	  |	94282.7372
    110010	  |	94762.1456
    100101	  |	96613.4529
    110011	  |	97071.1463
    000111	  |	120749.2172
    000110	  |	120945.0647
    101000	  |	124041.3991
    101001	  |	127515.5645
    011000	  |	130577.7115
    011001	  |	132698.1000
    001010	  |	140972.2754
    001011	  |	141919.8777
    110100	  |	142646.1812
    110101	  |	145358.1095
    100110